In [1]:
import itertools
 
def gera_pemutacoes():
  sequencia = 'ABCDEF'
 
  nums = list(sequencia)
  permutations = list(itertools.permutations(nums))
 
  return [''.join(permutation) for permutation in permutations]
permutacoes = gera_pemutacoes()
block_sizes_i = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
block_sizes_j = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]
block_sizes_k = [1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024, 2048]


In [2]:
def getNumeroAlgoritmo(permutacao, block_size_i, block_size_j, block_size_k):
  indice_sequencia = permutacoes.index(permutacao)
  indice_block_size_i = block_sizes_i.index(block_size_i)
  indice_block_size_j = block_sizes_j.index(block_size_j)
  indice_block_size_k = block_sizes_k.index(block_size_k)
  algoritmo = indice_sequencia + indice_block_size_i * 720 + indice_block_size_j * 7200 + + indice_block_size_k * 72000
  return algoritmo

def gera_comandos(sequencia):
  comandos = ''
  for comando in sequencia:
    match comando:
      case 'A': comandos += 'for (block_i = 0; block_i < n; block_i += block_size_i)\n'
      case 'B': comandos += 'for (block_j = 0; block_j < n; block_j += block_size_j)\n'
      case 'C': comandos += 'for (block_k = 0; block_k < n; block_k += block_size_k)\n'
      case 'D': comandos += 'for (i = 0; i < block_size_i; ++i)\n'
      case 'E': comandos += 'for (j = 0; j < block_size_j; ++j)\n'
      case 'F': comandos += 'for (k = 0; k < block_size_k; ++k)\n'
  return comandos

def gera_funcao(sequencia, block_size_i, block_size_j, block_size_k):
  codigo = f'''
void matrix_dgemm_2_{sequencia}_{block_size_i}_{block_size_j}_{block_size_k}(int n, double *restrict _C, double *restrict _A, double *restrict _B)
{{
#define A(i, j) _A[n * (i) + (j)]
#define B(i, j) _B[n * (i) + (j)]
#define C(i, j) _C[n * (i) + (j)]

int block_size_i = {block_size_i};
int block_size_j = {block_size_j};
int block_size_k = {block_size_k};

int block_i, block_j, block_k;
int i, j, k;

{gera_comandos(sequencia)}

C(block_i + i, block_j + j) += A(block_i + i, block_k + k) * B(block_k + k, block_j + j);
  
#undef A
#undef B
#undef C
}} 

'''
  return codigo


def gera_cases():
  conteudo = '' 
  for block_size_k in block_sizes_k:
    for block_size_j in block_sizes_j:
      for block_size_i in block_sizes_i:
        for sequencia in permutacoes:
          indice = getNumeroAlgoritmo(sequencia, block_size_i, block_size_j, block_size_k)
          conteudo += f'''
          case {indice}:
            matrix_dgemm_2_{sequencia}_{block_size_i}_{block_size_j}_{block_size_k}(n, _C, _A, _B);
            ret = true;
            break;

            '''
  return conteudo

def gera_cases_unico(block_size_i, block_size_j, block_size_k):
  conteudo = '' 
  for sequencia in permutacoes:
    indice = permutacoes.index(sequencia)
    conteudo += f'''
    case {indice}:
      matrix_dgemm_2_{sequencia}_{block_size_i}_{block_size_j}_{block_size_k}(n, _C, _A, _B);
      ret = true;
      break;

      '''
  return conteudo

def gera_matrix_which_dgemm(block_size_i, block_size_j, block_size_k):
  conteudo = f'''
bool matrix_which_dgemm(int algorithm, int n, double *restrict _C, double *restrict _A, double *restrict _B)
{{
  bool ret;
  switch (algorithm)
  {{
  
  {gera_cases_unico(block_size_i, block_size_j, block_size_k)}

  default:
    ret = false;
  }}
  return ret;
}}
'''
  return conteudo



In [3]:
def gera_matrix_c(block_size_i, block_size_j, block_size_k, matrix_h):
  conteudo = f'''#include "{matrix_h}"

#include <stdlib.h>

#define ABS(x) ((x > 0)? (x): -(x))


void matrix_fill_rand(int n, double *restrict _A)
{{
    #define A(i, j) _A[n*(i) + (j)]
	int i, j;

	for (i = 0; i < n; ++i)
        for (j = 0; j < n; ++j)
            A(i, j) = 10*(double) rand() / (double) RAND_MAX;

    #undef A
}}

bool matrix_eq(int n, double *restrict _A, double *restrict _B)
{{
    #define A(i, j) _A[n*(i) + (j)]
    #define B(i, j) _B[n*(i) + (j)]

    int i, j;

    for (i = 0; i < n; ++i)
        for (j = 0; j < n; ++j)
            if (ABS (A(i, j) - B(i, j)) > 1e-15)
                return false;

    return true;
    #undef A
    #undef B
}}

  '''    

  for sequencia in permutacoes:
    conteudo += gera_funcao(sequencia, block_size_i, block_size_j, block_size_k)

  conteudo += gera_matrix_which_dgemm(block_size_i, block_size_j, block_size_k)
  return conteudo


In [4]:
def gera_matrix_h(block_size_i, block_size_j, block_size_k):
  conteudo = '''
/* Operações com matrizes. */

/* Isso aqui embaixo é um header guard. Serve para evitar dupla-inclusão
 * deste cabeçalho
 */
#ifndef _MATRIX_H
#define _MATRIX_H

/* Para usar booleanos em C */
#include <stdbool.h>

#define FOR_EACH_DGEMM(i) for (i = 0; i < 3; ++i)

/* Encha a matriz A com dados aleatórios.
 * A: Matriz
 * n: número de linhas = número de colunas
 */
void matrix_fill_rand(int n, double *restrict _A);

/* Seleciona qual dgemm usar */
bool matrix_which_dgemm(int algorithm, int n, double *restrict _C, double *restrict _A, double *restrict _B);

/* Retorna true se as matrizes são iguais, e false c.c.*/
bool matrix_eq(int n, double *restrict _A, double *restrict _B);

'''
  for sequencia in permutacoes:
    conteudo +=  f'void matrix_dgemm_2_{sequencia}_{block_size_i}_{block_size_j}_{block_size_k}(int n, double *restrict _C, double *restrict _A, double *restrict _B);\n'

  conteudo += '#endif'
  return conteudo

In [5]:
def gera_main(block_size_i, block_size_j, block_size_k, matrix_h):
  conteudo = f'''
/* Arquivo principal.
 * Programa criado para facilitar a coleta de dados a partir de um script.
 * Para pegar o tempo, basta ler o número impresso pelo stdout.
 */

#include "{matrix_h}"
#include "time_extra.h"

#include <stdio.h>
#include <stdbool.h>
#include <stdint.h>
#include <stdlib.h>
#include <string.h>

#define INVALID_ALG  3 /* Parâmetro algorithm não inicializado */
#define INVALID_SIZE 0 /* Tamanho da matriz não incializado */

struct parsed_args
{{
    int matrix_size; /* tamanho da matriz */
    unsigned algorithm;   /* qual dgemm vai ser chamado? */
    unsigned block_size_i;
    unsigned block_size_j;
    unsigned block_size_k;
}};


/* Parseie os argumentos fornecidos pelo argv */
static struct parsed_args* parse_args(int argc, const char* argv[])
{{
    /* A keyword `static` faz com que o compilador aloque o conteúdo no
     * segmento .data, assim o dado não será destruido quando a função
     * retornar. Note também que isso não é thread-safe.
     */
    static struct parsed_args ret = {{
        .matrix_size = INVALID_SIZE,
        .algorithm = INVALID_ALG
    }};

    int i;

    for (i = 0; i < argc-1; ++i)
    {{
        if (!strcmp(argv[i], "--matrix-size"))
            ret.matrix_size = atoi(argv[i+1]);
        else if (!strcmp(argv[i], "--algorithm"))
            ret.algorithm = atoi(argv[i+1]);
        else if (!strcmp(argv[i], "--block_size_i"))
            ret.block_size_i = atoi(argv[i+1]);
        else if (!strcmp(argv[i], "--block_size_j"))
            ret.block_size_j = atoi(argv[i+1]);
        else if (!strcmp(argv[i], "--block_size_k"))
            ret.block_size_k = atoi(argv[i+1]);                        
    }}
    return &ret;
}}

/* Checa se os argumentos parseados são válidos */
static bool validate_args(const struct parsed_args* args)
{{
    if (args->matrix_size == INVALID_SIZE )
        return false;
    return true;
}}

static void print_usage_message()
{{
    const char* msg =
    "Parâmetros incorretos. Uso:\\n"
    "  main <ARGS>\\n"
    "onde:\\n"
    "  --matrix-size <NUM>     Tamanho da matriz quadrada. (n_linhas = n_colunas)\\n"
    "  --algorithm <NUM>       Número da implementação. (0 = dgemm_0, 1 = dgemm_1, 2 = dgemm_2)\\n"
    "\\n";

    fputs(msg, stdout);
}}

int main(int argc, const char* argv[])
{{
    struct parsed_args* args = parse_args(argc, argv);

    /* Aqui, restrict quer dizer que o apontador não terá "aliasing".
     * Procure por "pointer aliasing" no google.
     */
    double *restrict A; /* Matriz A */
    double *restrict B;
    double *restrict C;

	struct timeval t1, t2, t3;

    int n;

    if (!validate_args(args))
    {{
        print_usage_message();
        return 1;
    }}

    n = args->matrix_size;

    A = aligned_alloc(8, n*n*sizeof(*A)); /* Aloque memória com alinhamento de 8 bytes */
    B = aligned_alloc(8, n*n*sizeof(*B));
    C = aligned_alloc(8, n*n*sizeof(*C));

    if (!(A && B && C))
    {{
        printf("Seu computador não tem memória pra isso!\\n");
        return 2;
    }}

    matrix_fill_rand(n, A);
    matrix_fill_rand(n, B);

    memset(C, 0x00, n*n*sizeof(*A));

    gettimeofday(&t1, NULL);
    if (!matrix_which_dgemm(args->algorithm, n, C, A, B))
    {{
        printf("Erro: O algoritmo selecionado é invalido!\\n");
        return 3;
    }}
    gettimeofday(&t2, NULL);

    timeval_subtract(&t3, &t2, &t1);

    printf("%lu.%06lu\\n", t3.tv_sec, t3.tv_usec);

    free(A);
    free(B);
    free(C);

    return 0;
}}
  
'''
  return conteudo


In [1]:
def gera_build_sh(main_c, matrix_c):
  conteudo = f'''
echo {main_c}
gcc  -Wall -pedantic -g -O2 -c {main_c}  
echo {matrix_c}
gcc  -Wall -pedantic -g -O2 -c {matrix_c}  
gcc  -Wall -pedantic -g -O2 -o {main_c[:-2]} {main_c[:-2]}.o {matrix_c[:-2]}.o time_extra.o
'''
  return conteudo

def prepara_build():
  conteudoA = conteudoB = conteudoC = conteudoD  = f'#!/bin/bash\n'
  for block_size_k in block_sizes_k:
    for block_size_j in block_sizes_j:
      for block_size_i in block_sizes_i:
        # block_size_i = 2
        # block_size_j = 4
        # block_size_k = 8


        matrix_c = f'matrix_{block_size_i}_{block_size_j}_{block_size_k}.c'
        main_c = f'main_{block_size_i}_{block_size_j}_{block_size_k}.c'


        if block_size_k in [2, 4, 512]:
          conteudoA += gera_build_sh(main_c, matrix_c)
        elif block_size_k in [8, 16]:
          conteudoB += gera_build_sh(main_c, matrix_c)
        elif block_size_k in [32, 64, 1024]:
          conteudoC += gera_build_sh(main_c, matrix_c)
        elif block_size_k in [128, 256]:
          conteudoD += gera_build_sh(main_c, matrix_c)
        else:
          print(f"Alguma coisa deu errado! {block_size_k}")

  with open('build_A.sh', 'w',encoding="utf-8") as file:
    file.write(conteudoA)
  file.close()

  with open('build_B.sh', 'w',encoding="utf-8") as file:
    file.write(conteudoB)
  file.close()

  with open('build_C.sh', 'w',encoding="utf-8") as file:
    file.write(conteudoC)
  file.close()

  with open('build_D.sh', 'w',encoding="utf-8") as file:
    file.write(conteudoD)
  file.close()


# prepara_build()

In [7]:
from os.path import exists
import os

def apaga_arquivos_gerados():
  for block_size_k in block_sizes_k:
    for block_size_j in block_sizes_j:
      for block_size_i in block_sizes_i:
        matrix_c = f'matrix_{block_size_i}_{block_size_j}_{block_size_k}.c'
        matrix_h = f'matrix_{block_size_i}_{block_size_j}_{block_size_k}.h'
        main_c = f'main_{block_size_i}_{block_size_j}_{block_size_k}.c'
        main = f'main_{block_size_i}_{block_size_j}_{block_size_k}'
        if exists(matrix_c):
          os.remove(matrix_c) 
        if exists(matrix_h):
          os.remove(matrix_h) 
        if exists(main_c):
          os.remove(main_c)
        if exists(main):
          os.remove(main)
apaga_arquivos_gerados()

In [2]:
def gera_build_fix():
  conteudo = f'#!/bin/bash\n'
  for block_size_k in block_sizes_k:
    for block_size_j in block_sizes_j:
      for block_size_i in block_sizes_i:
        # block_size_i = 2
        # block_size_j = 4
        # block_size_k = 8

        matrix_c = f'matrix_{block_size_i}_{block_size_j}_{block_size_k}.c'
        main_c = f'main_{block_size_i}_{block_size_j}_{block_size_k}.c'

        conteudo += f'''
echo Consertando {main_c[:-2]}!!!
gcc  -Wall -pedantic -g -O2 -o {main_c[:-2]} {main_c[:-2]}.o {matrix_c[:-2]}.o time_extra.o\n

'''


  with open('build-fix.sh', 'w',encoding="utf-8") as file:
    file.write(conteudo)
  file.close()

  


# gera_build_fix()

In [6]:


def prepara_build_complementar():
  conteudo = f'#!/bin/bash\n'
  for block_size_k in block_sizes_k:
    for block_size_j in block_sizes_j:
      for block_size_i in block_sizes_i:

        # if block_size_k in [1, 2048] or block_size_j in  [1, 2048] or block_size_k in  [1, 2048]:
        if block_size_i in  [1, 2048]:

          matrix_c = f'matrix_{block_size_i}_{block_size_j}_{block_size_k}.c'
          main_c = f'main_{block_size_i}_{block_size_j}_{block_size_k}.c'

          conteudo += gera_build_sh(main_c, matrix_c)
          

  with open('build_E_complementar.sh', 'w',encoding="utf-8") as file:
    file.write(conteudo)
  file.close()

 


prepara_build_complementar()

In [7]:
def gera_codigo():
  
  for block_size_k in block_sizes_k:
    for block_size_j in block_sizes_j:
      for block_size_i in block_sizes_i:
  
  # block_size_i = 2
  # block_size_j = 4
  # block_size_k = 8

              
        matrix_c = f'matrix_{block_size_i}_{block_size_j}_{block_size_k}.c'
        matrix_h = f'matrix_{block_size_i}_{block_size_j}_{block_size_k}.h'
        main_c = f'main_{block_size_i}_{block_size_j}_{block_size_k}.c'
        
        conteudo = gera_matrix_c(block_size_i, block_size_j, block_size_k, matrix_h)

        with open(matrix_c, 'w',encoding="utf-8") as file:
          file.write(conteudo)
        file.close()

        conteudo = gera_matrix_h(block_size_i, block_size_j, block_size_k)
        with open(matrix_h, 'w',encoding="utf-8") as file:
          file.write(conteudo)
        file.close()

        conteudo = gera_main(block_size_i, block_size_j, block_size_k, matrix_h)
        with open(main_c, 'w',encoding="utf-8") as file:
          file.write(conteudo)
        file.close()

gera_codigo()

In [9]:
import optuna
import tqdm as notebook_tqdm
import subprocess

def run(sequencia, block_size_i, block_size_j, block_size_k):
  algoritmo = str(permutacoes.index(sequencia))
  timeout_s = 10  # em segundos
  cmd = [f'./main_{block_size_i}_{block_size_j}_{block_size_k}', '--matrix-size', '2048', '--algorithm', algoritmo]
  try:
    result = subprocess.run(cmd, shell=False, capture_output=True, timeout=timeout_s, universal_newlines=True)
    return float(result.stdout)
  except subprocess.TimeoutExpired as e:
    return float(100)
  

def objective(trial):
  sequencia = trial.suggest_categorical("sequence", permutacoes)
  block_size_i = trial.suggest_categorical("block_size_i", block_sizes_i)
  block_size_j = trial.suggest_categorical("block_size_j", block_sizes_j)
  block_size_k = trial.suggest_categorical("block_size_k", block_sizes_k)  
  
  return run(sequencia, block_size_i, block_size_j, block_size_k)

def tenta():
  study = optuna.create_study(
    storage="sqlite:///db.sqlite3",
    study_name="miniEP6-v4"#,
    load_if_exists=True
  )
  study.optimize(objective, n_trials=1000)
  print(f"Best value: {study.best_value} (params: {study.best_params})")




tenta()


[I 2023-05-09 22:21:56,525] A new study created in RDB with name: miniEP6-v4
[I 2023-05-09 22:22:04,002] Trial 0 finished with value: 7.183524 and parameters: {'sequence': 'FBADCE', 'block_size_i': 16, 'block_size_j': 512, 'block_size_k': 256}. Best is trial 0 with value: 7.183524.
[I 2023-05-09 22:22:14,131] Trial 1 finished with value: 100.0 and parameters: {'sequence': 'FEBACD', 'block_size_i': 128, 'block_size_j': 4, 'block_size_k': 256}. Best is trial 0 with value: 7.183524.
[I 2023-05-09 22:22:24,258] Trial 2 finished with value: 100.0 and parameters: {'sequence': 'FCBDAE', 'block_size_i': 4, 'block_size_j': 128, 'block_size_k': 32}. Best is trial 0 with value: 7.183524.
[I 2023-05-09 22:22:34,384] Trial 3 finished with value: 100.0 and parameters: {'sequence': 'BDFCEA', 'block_size_i': 512, 'block_size_j': 64, 'block_size_k': 256}. Best is trial 0 with value: 7.183524.
[I 2023-05-09 22:22:43,286] Trial 4 finished with value: 8.624237 and parameters: {'sequence': 'AFCDBE', 'block

KeyboardInterrupt: 

In [10]:
permutacoes.index('BDFACE')


186